In [9]:
%matplotlib inline

In [14]:
# Import python modules
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import netCDF4
import gc, sys, os
sys.path.append("../scripts")

import spatial_functions
import aem_utils
import netcdf_utils
import modelling_utils
import plotting_functions as plots
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
# Dash dependencies
import plotly.express as px
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.graph_objects as go


Import the LCI



In [2]:
# The actual inversoin data are stored on disk as netcdf files. NetCDF is an efficient format for storing 
# self-describing containerised data. 
# The implementation of netcdf for AEM line data was done by Alex Ip using his geophys_utils package.
# https://github.com/GeoscienceAustralia/geophys_utils/tree/master/geophys_utils


root = r"C:\Users\symin\OneDrive\Documents\GA\AEM\LCI"

# Define path to the netcdf file
infile = os.path.join(root, "SouthernStuart_WB_MGA53.nc")

# Create an instance
lci = aem_utils.AEM_inversion(name = 'Laterally Contrained Inversion (LCI)',
                              inversion_type = 'deterministic',
                              netcdf_dataset = netCDF4.Dataset(infile))

# As these inversions have already been gridded we will add these raster datasets to the instance using the
# load_lci_layer_grid() function. This function belongs to the AEM_inversion class.

# Directory in which the grids are located
infile = os.path.join(root, "grids\\SSC_LCI_layer_grids.p")

# Run function
lci.load_lci_layer_grids_from_pickle(infile)

# Create polylines
lci.create_flightline_polylines()

Create an instance of the garjmcmctdem inversion and probe the results using the same syntax as above.

In [3]:
# Path to netcdf file
infile = r"C:\Users\symin\OneDrive\Documents\GA\AEM\rjmcmc\SSC_vanilla_rjmcmc_pmaps.nc"


# Create instance
rj = aem_utils.AEM_inversion(name = 'GARJMCMCTDEM',
                             inversion_type = 'stochastic',
                             netcdf_dataset = netCDF4.Dataset(infile))

In [4]:
rj.data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    nlayers_min: 1
    nlayers_max: 20
    nsamples: 700000
    nchains: 4
    nburnin: 350000
    thinrate: 25
    min_log10_conductivity: -2.6989700043360187
    max_log10_conductivity: 0.6989700043360189
    min_depth: 0.0
    max_depth: 300.0
    value_parameterization: LOG10
    position_parameterization: LINEAR
    keywords: geophysics, airborne, AEM, conductivity, garjmcmcmtdem
    date_created: 2020-09-16 23:01:37.769828
    crs: GDA94 / MGA zone 53
    crs_geographic: GDA2020
    geospatial_east_min: 379423.38
    geospatial_east_max: 453810.25
    geospatial_east_units: m
    geospatial_north_min: 7348389.5
    geospatial_north_max: 7376633.0
    geospatial_north_units: m
    geospatial_vertical_min: 413.81
    geospatial_vertical_max: 601.86
    geospatial_vertical_units: m
    geospatial_lon_min: 133.81596794267762
    geospatial_lon_max: -23.720160830086574
    geospatial_lont_units: degr

In [5]:
# Now we have the lines we can grid the lci conductivity data onto vertical grids (known as sections)
# this is the easiest way to visualise the AEM conuctivity in 2-dimensions

# Assign the lci variables to grid
grid_vars = ['conductivity', 'data_residual', 'depth_of_investigation']


# Define the resolution of the sections
xres, yres = 40., 5.

# We will use the lines from the rj

lines = rj.data['line'][:]

# Define the output directory if saving the grids as hdf plots

hdf5_dir = r"C:\Temp\SSC_hdf5"

# if the directory doesn't exist, then create it
if not os.path.exists(hdf5_dir):
    os.mkdir(hdf5_dir)  

# Gridding takes a few minutes so I pre-gridded them for you. The lci.grid_sections()
# function below will do the gridding for you. Instead we will use the load_sectoin_from_file()
# function, which loads hdf5 files produced using the grid_sections() function


#lci.grid_sections(variables = grid_vars, lines = lines, xres = xres, yres = yres,
#                  return_interpolated = True, save_hdf5 = True, hdf5_dir = hdf5_dir)

lci.load_sections_from_file(hdf5_dir, grid_vars, lines = lines)

In [6]:
# Grid the rj sections

# Assign the lci variables to grid
grid_vars = ['conductivity_p10', 'conductivity_p50', 'conductivity_p90', 'interface_depth_histogram']

# Define the resolution of the sections
xres, yres = 100., 2.

# We will use the lines from the rj

lines = np.unique(rj.data['line'][:].astype('int'))

# Define the output directory if saving the grids as hdf plots

hdf5_dir = r"C:\Temp\SSC_hdf5_rj"

# if the directory doesn't exist, then create it
if not os.path.exists(hdf5_dir):
    os.mkdir(hdf5_dir)  

#rj.grid_sections(variables = grid_vars, lines = lines, xres = xres, yres = yres,
#                  return_interpolated = True, save_hdf5 = True, hdf5_dir = hdf5_dir)

rj.load_sections_from_file(hdf5_dir, grid_vars, lines = lines)


In [26]:
# Using this gridding we find the distance along the line for each site
# Iterate through the lines
rj.distance_along_line = {}

for lin in lines:
    # Get a line mask
    line_mask = netcdf_utils.get_lookup_mask(lin, rj.data)
    # get the coordinates
    line_coords = rj.coords[line_mask]

    dists = spatial_functions.xy_2_var(lci.section_data[lin],
                                      line_coords,
                                      'grid_distances')
    # Add a dictionary with the point index distance along the line to our inversion instance
    rj.distance_along_line[lin] = pd.DataFrame(data = {"point_index": np.where(line_mask)[0],
                                                       "distance_along_line": dists,
                                                       'fiducial': rj.data['fiducial'][line_mask]}
                                               ).set_index('point_index')

In [27]:
rj.distance_along_line[lin]

,distance_along_line,fiducial
point_index,,
54,11040.0,694851.5
136,360.0,695171.5
148,11200.0,694846.5
155,12000.0,694821.5
161,6240.0,694996.5
...,...,...
1894,8040.0,694941.5
1926,6560.0,694986.5
1948,9880.0,694886.5


# 2. Visualise the data


# 2.1 Map plot

Our first standard plot is an plot of gridded conductivity with the points inverted with GARJMCMCTDEM
plotted as points.

# 3 Setup your model

Now we want to create a model boundary object for our interpreted surface. The easiest way to manage this is to have a separate instance the module boundary for every interface. This will allow us to produce eggs ready output files as we go.


In [28]:
# Create an modelled boundary instance

headings = ["inversion_name",'X', 'Y', 'ELEVATION', "DEM", "DEPTH", "UNCERTAINTY", "Type",
            "BoundaryNm", "BoundConf", "BasisOfInt", "OvrConf", "OvrStrtUnt", "OvrStrtCod", "UndStrtUnt",
           "UndStrtCod", "WithinType", "WithinStrt", "WithinStNo", "WithinConf", "InterpRef",
            "Comment", "SURVEY_LINE", "Operator"]

interp_file = r"C:\temp\Mereenie-Pertnjara_interface_interpreted_points.csv"

MP_surface = modelling_utils.modelled_boundary(name = 'Mereenie-Pertnjara interface',
                                               outfile_path = interp_file,
                                               interpreted_point_headings = headings)
MP_surface.interpreted_points = pd.read_csv(interp_file)
# Define your surface
surface = MP_surface

# Assign attributes base on what you want to be
# entered into the eggs database
surface.Type = "BASE_Cenozoic_TOP_Paleozoic"
surface.OvrStrtUnt = "Pertnjara Formation"
surface.OvrStrtCod = 15098
surface.UndStrtUnt = "Mereenie Sandstone"
surface.UndStrtCod = 11667
surface.Inversion_name = "garjmcmtdem"
surface.BoundConf = "M"
surface.BasisOfInt = "IAEM"
surface.OvrConf = "M"
surface.InterpRef = ""
surface.Comment = ""
surface.Operator = "Neil Symington"
surface.WithinType = ""
surface.WithinStrt = ""
surface.WithinStNo = ""
surface.WithinConf = ""

line_options = []

for l in lines:
     line_options.append({'label': str(l), 'value': l})


In [30]:
def subset_df_by_line(df_, line, line_col = 'SURVEY_LINE'):
    mask = df_[line_col] == line
    return df_[mask]


def interp2scatter(line, gridded_data, interpreted_points, easting_col = 'X',
                   northing_col = 'Y', elevation_col = 'ELEVATION',
                   line_col = 'SURVEY_LINE'):

    utm_coords = np.column_stack((gridded_data[line]['easting'],
                                  gridded_data[line]['northing']))

    df_ = subset_df_by_line(interpreted_points, line, line_col = line_col)

    dist, inds = spatial_functions.nearest_neighbours(df_[[easting_col,northing_col]].values,
                                                      utm_coords, max_distance=100.)

    grid_dists = gridded_data[line]['grid_distances'][inds]
    elevs = df_[elevation_col].values
    fids = df_['fiducial'].values

    return  grid_dists, elevs, fids

def layer_point_prob_plot(section_data, line):
    # Flatten into x,z, probability
    c = section_data[line]['interface_depth_histogram'].flatten()


    x = np.tile(section_data[line]['grid_distances'],
                len(section_data[line]['grid_elevations']))
    y = np.repeat(section_data[line]['grid_elevations'],
                  len(section_data[line]['grid_distances']))

    x = x[np.isfinite(c)]
    y = y[np.isfinite(c)]
    c = c[np.isfinite(c)]

    # convert to probability
    c = c/np.max(c)

    return x, y, c

def dash_section(line, df_interp, colours, section_kwargs):
    # Create the grid
    if section_kwargs['section_plot'] == "lci":
        section_data = lci.section_data
        fig = px.imshow(img = np.log10(section_data[line]['conductivity']),
                        zmin = np.log10(section_kwargs['vmin']),
                        zmax = np.log10(section_kwargs['vmax']),
                        x = section_data[line]['grid_distances'],
                        y = section_data[line]['grid_elevations'],
                        color_continuous_scale=section_kwargs['cmap'], aspect = 'auto',
                        labels=dict(x="Distance along line", y="elevation (mAHD)"),
                        template='plotly'
        )
    elif section_kwargs['section_plot'] == "rj-p50":
        section_data = rj.section_data

        fig = px.imshow(np.log10(section_data[line]['conductivity_p50']),
                        zmin = np.log10(section_kwargs['vmin']),
                        zmax = np.log10(section_kwargs['vmax']),
                        x = section_data[line]['grid_distances'],
                        y = section_data[line]['grid_elevations'],
                        color_continuous_scale=section_kwargs['cmap'],
                        aspect = 'auto',
                        labels=dict(x="Distance along line", y="elevation (mAHD)"),
                        template='plotly'
        )
    elif section_kwargs['section_plot'] == 'rj-conf':
        section_data = rj.section_data

        confidence = plots.percentiles2pnci(section_data[line]['conductivity_p10'],
                                            section_data[line]['conductivity_p90'],
                                            upper_threshold = 0.99,
                                            lower_threshold = 0.01)

        fig = px.imshow(confidence,
                        zmin = 0.1,
                        zmax = 0.9,
                        x = section_data[line]['grid_distances'],
                        y = section_data[line]['grid_elevations'],
                        color_continuous_scale='GnBu',
                        aspect = 'auto',
                        labels=dict(x="Distance along line", y="elevation (mAHD)"),
                        template='plotly'
        )

    elif section_kwargs['section_plot'] == "rj-lpp":
        section_data = rj.section_data

        x,y,c = layer_point_prob_plot(section_data, line)

        fig = px.scatter(x = x, y= y, color = c,
                         color_continuous_scale='Greys',
                         labels=dict(x="Distance along line", y="elevation (mAHD)"),
                         template='plotly')

    # Add the elevation
    fig.add_trace(go.Scatter(x = section_data[line]['grid_distances'],
                             y = section_data[line]['elevation'],
                             line=dict(color='black', width=3),
                             showlegend = False, hoverinfo = None))

    # Now we add the rjmcmc sites to the section
    df_rj_sites = rj.distance_along_line[line]

    labels = ["fiducial = " + str(x) for x in df_rj_sites['fiducial']]

    fig.add_scatter(x = df_rj_sites['distance_along_line'].values,
                    y = 20. +np.max(section_data[line]['elevation'])*np.ones(shape = len(df_rj_sites),
                                                                        dtype = np.float),
                    mode = 'markers',
                    hovertext = labels)


    if np.logical_or(section_kwargs['section_plot'] == "rj-p50", section_kwargs['section_plot'] == "lci"):
        # Get the ticks
        tickvals = np.linspace(np.log10(section_kwargs['vmin']),
                                np.log10(section_kwargs['vmax']),
                                5)

        ticktext = [str(np.round(x,3)) for x in 10**tickvals]

        fig.update_layout(coloraxis_colorbar=dict(
        title="conductivity",
        tickvals=tickvals,
        ticktext=ticktext,
        ))

    interpx, interpz, fids = interp2scatter(line, section_data, df_interp)

    if len(interpx) > 0:
        labels = ["fiducial = " + str(x) for x in fids]

        fig.add_scatter(x = interpx,
                        y = interpz,
                        mode = 'markers',
                        hovertext = labels,
                        marker = {"color": colours})

    # Reverse y-axis
    fig.update_yaxes(autorange=True)

    return fig



In [31]:
# you may need extra dependencies to run these cells
import dash_bootstrap_components as dbc

section_kwargs = {'colourbar_label': 'Conductivity (S/m)',
                  'vmin': 0.01,
                  'vmax': 1.,
                  'cmap': 'jet'}

stylesheet = "https://codepen.io/chriddyp/pen/bWLwgP.css"
app = JupyterDash(__name__, external_stylesheets=[stylesheet])

app.layout = html.Div([
    html.Div(
                [
                    html.Div(html.H1("AEM interpretation dash board"),
                             className= "four columns"),
                    html.Div([html.H4("Select section"),
                             dcc.Dropdown(id = "section_dropdown",
                                            options=[
                                                    {'label': 'laterally constrained inversion',
                                                     'value': 'lci'},
                                                    {'label': 'garjmcmctdem - p50',
                                                     'value': 'rj-p50'},
                                                   {'label': 'garjmcmcm - certainty',
                                                     'value': 'rj-conf'},
                                                    {'label': 'garjmcmctdem - layer probability',
                                                     'value': 'rj-lpp'}],
                                            value="lci"),

                             ],className = "four columns"),
                    html.Div([html.H4("Select line"),
                             dcc.Dropdown(id = "line_dropdown",
                                            options=line_options,
                                            value= int(line_options[0]['label'])),
                             ],className = "four columns")
                ], className = 'row'
            ),
    html.Div(html.Div(id='section')),
    html.Div([html.Div(
        dash_table.DataTable(id='interp_table',
                                    css=[{'selector': '.row', 'rule': 'margin: 0'}],
                                    #columns=[{"name": i, "id": i} for i in df.columns],
                                    #data=df.to_dict('records'),
                                    fixed_columns={ 'headers': True},#, 'data': 1 },
                                    sort_action="native",
                                    sort_mode="multi",
                                    row_selectable="multi",
                                    row_deletable=True,
                                    selected_columns=[],
                                    selected_rows=[],
                                    style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                               'height': '40px'},
                                    style_cell={
                                                 'backgroundColor': 'rgb(50, 50, 50)',
                                                 'color': 'white',
                                                 'minHeight': '50px',
                                                 'minWidth': '0px', 'maxWidth': '800px',
                                                 'whiteSpace': 'normal',
                                                 'font-size': '12px'
                                             },
                                  style_table={
                                              'maxHeight': '1000px',
                                              'overflowY': 'scroll',
                                              'maxWidth':  '1000px',
                                              'overflowX': 'scroll'})
                                        , className = "four columns"),
        html.Div(html.Div(id='pmap_plot'), className = "four columns"),
        html.Div(html.Div(id='poly_line_plot'), className = "four columns")]

             ),

])

@app.callback(
    [Output('interp_table', 'data'),
    Output('interp_table', 'columns')],
    [Input("line_dropdown", 'value')])
def update_data_table(value):
    df_ss = subset_df_by_line(surface.interpreted_points,
                              line = value)
    return df_ss.to_dict('records'), [{"name": i, "id": i} for i in df_ss.columns]

@app.callback(
    Output('section', "children"),
    [Input("line_dropdown", 'value'),
     Input("section_dropdown", 'value'),
     Input('interp_table', "derived_virtual_data"),
     Input('interp_table', "derived_virtual_selected_rows")])
def update_graphs(line, section_plot, rows, derived_virtual_selected_rows):
    # When the table is first rendered, `derived_virtual_data` and
    # `derived_virtual_selected_rows` will be `None`. This is due to an
    # idiosyncrasy in Dash (unsupplied properties are always None and Dash
    # calls the dependent callbacks when the component is first rendered).
    # So, if `rows` is `None`, then the component was just rendered
    # and its value will be the same as the component's dataframe.
    # Instead of setting `None` in here, you could also set
    # `derived_virtual_data=df.to_rows('dict')` when you initialize
    # the component.
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []

    dff = surface.interpreted_points if rows is None else pd.DataFrame(rows)

    colours = ['pink' if i in derived_virtual_selected_rows else 'white'
              for i in range(len(dff))]

    section_kwargs['section_plot'] = section_plot

    fig = dash_section(line, dff, colours, section_kwargs)

    return [
        dcc.Graph(
            id='graph',
            figure=fig
            ),
    ]

app.run_server(mode='external', port=8060)



Dash app running on http://127.0.0.1:8060/


In [10]:
# if needed
#app._terminate_server_for_port("localhost", 8050)
#app = None

KeyError: 'conductivity-p50'

{100101: {'conductivity_p10': array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       ...,
       [       nan,        nan,        nan, ..., 0.01292442, 0.00959156,
        0.0043295 ],
       [       nan,        nan,        nan, ..., 0.01292442, 0.00959156,
        0.0043295 ],
       [       nan,        nan,        nan, ..., 0.0113373 , 0.00854614,
        0.00423604]], dtype=float32), 'conductivity_p50': array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       ...,
       [       nan,        nan,        nan, ..., 0.273789  , 0.